In [1]:
from cd.ds import *

In [2]:
ds=WV_S1(Path('DS/WV2_Site1/'),64)



In [3]:
trainds,testds,valds=split(ds,0.1,0.1)
trainds.augment=True

In [4]:
from torch.utils.data import DataLoader
train_loader = DataLoader(trainds, batch_size=10, shuffle=True, num_workers=4, pin_memory=True)


In [6]:
from cd.models.fcef.siamunet_diff import SiamUnet_diff
model=SiamUnet_diff(3,2)
model=model.float()

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('Number of trainable parameters:', count_parameters(model))

Number of trainable parameters: 1350146


In [8]:
from torch import nn
criterion = nn.NLLLoss() 

optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-4)
#     optimizer = torch.optim.Adam(net.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.95)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
import time
def train_epoch(train_loader, model, criterion, optimizer):
    """
    train one epoch
    :param train_loader: DataLoader
    :param model: model
    :param criterion: loss function
    :param optimizer: optimizer
    returns (losses,data_time,batch_time)
    """
    model.train()
    start = time.time()
    losses = []
    for i, (im1,im2,cm) in enumerate(train_loader):
        # Move to device
        
        im1, im2, cm = im1.float().to(device), im2.float().to(device),cm.long().to(device)
        # Forward
        output = model(im1, im2)

        # Loss
        loss = criterion(output, cm)


        # Backward
        optimizer.zero_grad()
        loss.backward()


        # Update model
        optimizer.step()
        print(loss)
    losses.append(loss)

In [22]:
def validate():
    pass
def checkpoint():
    pass

In [28]:
from tqdm import tqdm
def train(train_loader, model, criterion, optimizer,nepochs):
    for epoch in tqdm(range(nepochs)):
        train_epoch(train_loader, model, criterion, optimizer)
        validate()
        checkpoint()
    


In [29]:
train(train_loader, model, criterion, optimizer,1)

  0%|          | 0/1 [00:00<?, ?it/s]

tensor(0.3797, grad_fn=<NllLoss2DBackward>)
tensor(0.5782, grad_fn=<NllLoss2DBackward>)
tensor(0.3297, grad_fn=<NllLoss2DBackward>)
tensor(0.5319, grad_fn=<NllLoss2DBackward>)
tensor(0.5243, grad_fn=<NllLoss2DBackward>)
tensor(0.5324, grad_fn=<NllLoss2DBackward>)
tensor(0.4372, grad_fn=<NllLoss2DBackward>)
tensor(0.3850, grad_fn=<NllLoss2DBackward>)
tensor(0.3793, grad_fn=<NllLoss2DBackward>)
tensor(0.5401, grad_fn=<NllLoss2DBackward>)
tensor(0.3257, grad_fn=<NllLoss2DBackward>)
tensor(0.2696, grad_fn=<NllLoss2DBackward>)
tensor(0.2982, grad_fn=<NllLoss2DBackward>)
tensor(0.2257, grad_fn=<NllLoss2DBackward>)
tensor(0.3208, grad_fn=<NllLoss2DBackward>)
tensor(0.3434, grad_fn=<NllLoss2DBackward>)
tensor(0.2710, grad_fn=<NllLoss2DBackward>)
tensor(0.4306, grad_fn=<NllLoss2DBackward>)
tensor(0.6379, grad_fn=<NllLoss2DBackward>)
tensor(0.3416, grad_fn=<NllLoss2DBackward>)
tensor(0.2618, grad_fn=<NllLoss2DBackward>)
tensor(0.4026, grad_fn=<NllLoss2DBackward>)
tensor(0.3868, grad_fn=<NllLoss2

  0%|          | 0/1 [02:28<?, ?it/s]


KeyboardInterrupt: 